# Data Extractor

### Libraries and Costants

In [ ]:
import os
from bs4 import BeautifulSoup
import json
from lxml import etree
import logging
import re
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

SOURCES_DIR = 'sources/'
EXTRACTION_DIR = 'extraction/'

if not os.path.exists(EXTRACTION_DIR):
    os.makedirs(f'./{EXTRACTION_DIR}', exist_ok=True)

In [ ]:
def salva_dati_in_json(dati_estratti, article_id, extraction_path):
    tabelle = dati_estratti
    output_file_path = f"{extraction_path}/{article_id}.json"
    os.makedirs(extraction_path, exist_ok=True)
    with open(output_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(tabelle, json_file, ensure_ascii=False, indent=2)

In [ ]:
def estrai_dati_da_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    parser = etree.HTMLParser()
    tree = etree.fromstring(content, parser)

    tabelle = {}
    table_counter = 0

    figure_with_tables = tree.xpath('//figure[.//table]')

    for figure in figure_with_tables:
        try:
            try:
                table_id = figure.xpath("@id")[0]
            except:
                table_id = figure.xpath('ancestor::div[2]/@id')[0]
            table = figure.xpath('.//table')[0]
            table_counter += 1
            table_key = f"id_table_{table_counter}"

            caption = figure.xpath('.//figcaption//text()')
            caption_text = '' 

            if caption:
                caption_text = ''.join([c.strip() for c in caption]).replace('  ', ' ')

            dati_tabella = []
            rows = table.xpath('.//tr[position()>1]')
            for row in rows:
                cols = row.xpath('.//td')
                dati_row = [etree.tostring(col, encoding='unicode', method='html') for col in cols]
                dati_tabella.append(dati_row)

            references = tree.xpath(f"//p[a/@href = '#{table_id}']")
            references_text = [ref.xpath('string(.)').replace('\n', '').strip() for ref in references]   #elimina /n ma introduce spazi bianchi
            
            note_a_pie_di_pagina = []
            footnotes = figure.xpath(".//span[contains(@id, 'footnote') and contains(@class, 'ltx_text')]")
            for footnote in footnotes:
                logging.info(f"footnote found: {footnote} at table: {table_id}")

                footnote_text = " ".join(footnote.xpath(".//text()")).strip()
                if footnote_text and footnote_text != " ":
                    footnote_text = footnote_text.replace("\n", " ")
                    note_a_pie_di_pagina.append(footnote_text)

            tabelle[table_key] = {
                "caption": caption_text,
                "table": dati_tabella,
                "footnotes": note_a_pie_di_pagina,
                "references": references_text,
            }

        except Exception as e:
            logging.error(f"Error processing figure in {file_path}: {e}")
            logging.error(f"Figure content: {etree.tostring(figure, encoding='unicode', pretty_print=True)}")
        
    return tabelle

Punto di lancio del codice

In [ ]:
# Itera attraverso i file nella cartella 'articoli' e estrae le tabelle, didascalie, note e riferimenti
total_tables = 0
total_captions = 0
total_footnotes = 0
total_references = 0
articolo_counter = 0

#questo é perché nel salvataggio dei file ho creato cartella articoli dentro a sources
for filename in os.listdir(SOURCES_DIR):
    file_path = os.path.join(SOURCES_DIR, filename)
    article_id = filename.split('_')[2].replace('.html', '')  # Estrae l'ID dal nome del file (primo elemento prima del punto)

    articolo_counter += 1  # Incrementa il contatore
    logging.info(f"Estraendo dati dall'articolo {articolo_counter}: {filename}...")

    # Estrai i dati dal file HTML
    dati_estratti = estrai_dati_da_file(file_path)

    # Update statistics counters
    for table_data in dati_estratti.values():
        total_tables += 1
        if table_data["caption"]:
            total_captions += 1
        total_footnotes += len(table_data["footnotes"])
        total_references += len(table_data["references"])

    # Salva i dati in JSON
    salva_dati_in_json(dati_estratti, article_id, EXTRACTION_DIR)
        
# Print statistics
logging.info("\n--- Extraction Statistics ---")
logging.info(f"Total Articles Processed: {articolo_counter}")
logging.info(f"Total Tables Found: {total_tables}")
logging.info(f"Total Captions Found: {total_captions}")
logging.info(f"Total Footnotes Found: {total_footnotes}")
logging.info(f"Total References Found: {total_references}")

# Valutazione

### Numero di tabelle estratte sul numero di tabelle totali

In [1]:
def evaluate():
    filepaths = os.listdir(SOURCES_DIR)
    totale_html = 0
    totale_json = 0
    pattern = re.compile(r'[a-zA-Z]\d+\.T(\d+)')

    for filename in filepaths:
        article_id = filename.split('_')[2].replace('.html', '')
        json_filename = f"{article_id}.json"
        json_filepath = os.path.join(EXTRACTION_DIR, json_filename)

        with open(os.path.join(SOURCES_DIR, filename), 'r', encoding='utf-8') as file, open(json_filepath, 'r', encoding='utf-8') as json_file:
            html_content = file.read()
            soup = BeautifulSoup(html_content, 'html.parser')

            ty_values = set()
            for tag in soup.find_all(id=pattern):
                match = pattern.search(tag['id'])
                if match:
                    ty_values.add(match.group())

            json_data = json.load(json_file)
            tables_in_html = len(ty_values)
            tables_in_json = len(json_data)

            totale_html += tables_in_html
            totale_json += tables_in_json
            if tables_in_json != tables_in_html:
                logging.warning(f"Found {tables_in_json}/{tables_in_html} tables (JSON/HTML) in {filename}")
            else:
                logging.info(f"Found: {tables_in_json}/{tables_in_html} tables (JSON/HTML) in {filename}")

    logging.info(f"Found: {totale_json}/{totale_html} tables")


In [ ]:
evaluate()

Confrontando le tabelle estratte con l'xPath: `//figure[.//table]` con le tabelle trovate nell'html cercando gli `id` contenenti la lettera **T** seguita da un numero, notiamo che la nostra assunzione è errata. Questo perché non sempre le tabelle sono formattate tramite un tag `<table>`, ad esempio nell'articolo 2309.17288 nessuna delle tre tabelle presenti compare in un tag `<table>`. \
\
Abbiamo quindi trovato **1879** tabelle tramite l'xPath su un totale di **1914** tabelle  

Riscrivo l'estrazione con un nuovo xpath più preciso

In [ ]:
def estrai_dati_da_file_v2(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    parser = etree.HTMLParser()
    tree = etree.fromstring(content, parser)

    tabelle = {}
    table_counter = 0

    elements = tree.xpath('//*[starts-with(@id, "S") or starts-with(@id, "A")][contains(@id, ".T") and contains(@class, "ltx_table")]')
    for element in elements:
        try:
            table_counter += 1
            table_id = element.attrib['id']
            if element.tag != 'figure':
                element = element.xpath('.//figure')[0]
            table = element.xpath('.//*[contains(@class, "ltx_tabular")]')[0]
            table_key = f"id_table_{table_counter}"
            caption = element.xpath('.//figcaption[contains(@class, "ltx_caption")]/text()')
            caption_text = ''

            if caption:
                caption_text = ''.join([c.strip() for c in caption]).replace('  ', ' ')
            
            rows = table.xpath('.//*[contains(@class, "ltx_tr")]')
            dati_tabella = []
            
            for row in rows:
                dati_tabella.append([etree.tostring(col, encoding='unicode', method='html',pretty_print=True) for col in row.xpath('.//*[contains(@class, "ltx_td")]')])
            
            references = tree.xpath(f"//p[a/@href = '#{table_id}']")
            references_text = [ref.xpath('string(.)').replace('\n', '').strip() for ref in references]   #elimina /n ma introduce spazi bianchi
                
            note_a_pie_di_pagina = []
            footnotes = element.xpath(".//span[contains(@id, 'footnote') and contains(@class, 'ltx_text')]")
            for footnote in footnotes:
                footnote_text = " ".join(footnote.xpath(".//text()")).strip()
                if footnote_text and footnote_text != " ":
                    footnote_text = footnote_text.replace("\n", " ")
                    note_a_pie_di_pagina.append(footnote_text)


            tabelle[table_key] = {
                "caption": caption_text,
                "table": dati_tabella,
                "footnotes": note_a_pie_di_pagina,
                "references": references_text,
                }

        except Exception as e:
            logging.error(f"Error processing figure in {file_path}: {e.with_traceback(None)}")
            logging.error(f"Figure content: {etree.tostring(element, encoding='unicode', pretty_print=True)}")

    return tabelle

In [ ]:
# Itera attraverso i file nella cartella 'articoli' e estrae le tabelle, didascalie, note e riferimenti
total_tables = 0
total_captions = 0
total_footnotes = 0
total_references = 0
articolo_counter = 0

#questo é perché nel salvataggio dei file ho creato cartella articoli dentro a sources
for filename in os.listdir(SOURCES_DIR):
    file_path = os.path.join(SOURCES_DIR, filename)
    article_id = filename.split('_')[2].replace('.html', '')  # Estrae l'ID dal nome del file (primo elemento prima del punto)

    articolo_counter += 1  # Incrementa il contatore
    logging.info(f"Estraendo dati dall'articolo {articolo_counter}: {filename}...")

    # Estrai i dati dal file HTML
    dati_estratti = estrai_dati_da_file_v2(file_path)

    # Update statistics counters
    for table_data in dati_estratti.values():
        total_tables += 1
        if table_data["caption"]:
            total_captions += 1
        total_footnotes += len(table_data["footnotes"])
        total_references += len(table_data["references"])

    # Salva i dati in JSON
    salva_dati_in_json(dati_estratti, article_id, EXTRACTION_DIR)
        
# Print statistics
logging.info("\n--- Extraction Statistics ---")
logging.info(f"Total Articles Processed: {articolo_counter}")
logging.info(f"Total Tables Found: {total_tables}")
logging.info(f"Total Captions Found: {total_captions}")
logging.info(f"Total Footnotes Found: {total_footnotes}")
logging.info(f"Total References Found: {total_references}")

In [ ]:
evaluate()

Considerazioni sui risultati ottenuti:\
Alcune tabelle non sono trovate mediante xpath, anche se esso è molto più selettivo e preciso del precedente, poiché i documenti HTML sono malformati (es.):
* il documento 2405.15145 che ha la prima tabella separata dal tag `<figure>` 
* il documento 2310.02071 che costruisce una tabella attraverso il tag `<svg>` il quale è renderizzato con errori sul documento.